In [ ]:
from llm.LLMModel import *
from ner.llm_ner.prompt_techniques.pt_abstract import PromptTechnique
from ner.llm_ner.prompt_techniques.pt_discussion import PT_OutputList
from ner.llm_ner.prompt_techniques.pt_gpt_ner import PT_GPT_NER
from ner.llm_ner.prompt_techniques.pt_gpt_ner import PT_GPT_NER
from ner.llm_ner.prompt_techniques.pt_wrapper import PT_Wrapper
from ner.llm_ner.few_shots_techniques import *
from ner.llm_ner.ResultInstance import load_all_results, load_result
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

%reload_ext autoreload
%autoreload 2

In [ ]:
df_results, raw_results= load_all_results(root_directory = "ner/saves/results/ontonote5/")
df = df_results.sort_values(['f1_mean', 'prompt_technique', 'nb_few_shots'], ascending = False)
df[df['model'].str.contains("few-shots")]

In [ ]:
for i, res3  in enumerate(raw_results) :
    res3 :ResultInstanceWithConfidenceInterval = res3
    nb_instance = dict['len_data_test']* dict['nb_test_run']
    dict =res3.get_dict() 
    time_elapsed = sum([res.get_dict()['elapsed_time'] for res in res3.res_insts])/nb_instance
    if dict['model'] == 'chat-gpt-3.5' and 'ft' not in dict['model'] :
        print(i, time_elapsed, nb_instance,  dict['nb_few_shots'], dict['prompt_technique'], dict)

In [ ]:
df_results[1][7].res_insts[0].analyse_results()

In [ ]:
df_results = load_all_results(root_directory = "ner/saves/results/ontonote5/")
df_results = df_results[df_results['prompt_technique'] == 'discussion']
df_results = df_results[df_results['few_shot_tecnique'] == 'sentence']
df_results.sort_values(['nb_few_shots', 'model'])

In [ ]:
df_to_show = df_results[~df_results['model'].str.contains('ft')][['f1_mean', 'f1_conf_inter', 'prompt_technique',
       'few_shot_tecnique', 'nb_few_shots', 'precision', 'plus_plus']]

df_res = df_to_show[df_to_show['precision'] == '300']
df_res['tech_name'] = df_res.apply(lambda row :f"With {row['nb_few_shots']} few_shots {'and ++' if row['plus_plus']  else ''}", axis = 1)
df_res

## Showing prompts 

In [ ]:
LLMModel.show_prompts()

## Looking at results of confidence with the tagger

In [ ]:
from ner.utils import load

res = load ("ner/saves/results/conll2003_cleaned/mistral-7b-v0.1/multi_prompt-get-entities-tagger/sentence_3_None_1538_50_1_False.pkl")

cm, f1, precision, recall, y_true, y_pred, y_conf= res.res_insts[0].get_scores(with_y = True)
conf = []
for sent in res.res_insts[0].results:
    conf.extend([s[2] for s in sent])

len(y_conf), len(y_true), len(y_pred)
# [i for i, tag in enumerate(y_pred) if tag == 'None']
right = [y_true[i] == y_pred[i] for i in range(len(y_true))]
rand = [i for i in range(len(right))]

In [ ]:
from collections import Counter

# Example list
my_list = list(zip(y_conf, right))

# Use Counter to count occurrences
entry_counts = Counter(sorted(my_list))

# Print the counts
for entry, count in entry_counts.items():
    print(f'{entry}: {count} times')

In [ ]:
import seaborn as sns
df_conf = pd.DataFrame(np.array([right, y_conf]).T, columns = ['Right', 'Confidence'])

df_conf = pd.DataFrame(df_conf.groupby(['Right', 'Confidence'])['Right'].count()).rename(columns = {'Right' : "count"}).reset_index()
sns.set(rc={'figure.figsize':(10,10)})
order = ['high', 'medium-high', 'medium', 'medium-low', 'low']
sns.catplot(data=df_conf, x="Confidence", y="count", hue="Right", kind="bar",aspect=1.5, order = order)


## Analysing specific result

In [ ]:
model  = "mistral-7b-v0.1"
result = load_result(model, PT_OutputList.name(), FST_Sentence.name(), nb_few_shots= 10, with_precision = "10_True")

for r in result.res_insts :
    r.show_cm()
    r.analyse_results()

In [ ]:
from ner.Datasets.Conll2003Dataset import Conll2003Dataset
from ner.Datasets.MyDataset import MyDataset
data = MyDataset.my_load_dataset(dataset=Conll2003Dataset, split = 'test', cleaned= True)
[(d['text'], d['spans']) for d in data if str(d['spans']) == """[['Switzerland', 'LOC'], ["Alfonse D'Amato", 'PER'], ['U.S. Senate Banking Committee', 'ORG'], ['Poland', 'LOC'], ['Polish', 'MISC'], ['Swiss', 'MISC'], ['Poland', 'LOC']]"""]